## Analyze A/B Test Results

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [73]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [74]:
df = pd.read_csv('ab_data.csv')
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


b. Use the below cell to find the number of rows in the dataset.

In [75]:
len(df)

294478

c. The number of unique users in the dataset.

In [76]:
df.user_id.nunique()

290584

d. The proportion of users converted.

In [77]:
(len(df.query('converted==1'))/len(df))*100

11.96591935560551

e. The number of times the `new_page` and `treatment` don't line up.

In [78]:
gr1 = len(df.query('group!="treatment" and landing_page=="new_page"')) #if langing page is new page 
gr2 = len(df.query('group!="control" and landing_page=="old_page"'))# if langing page is old page
gr=gr1+gr2
gr

3893

f. Do any of the rows have missing values?

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [80]:
#we will copy the data frame in a new dataframe
df2 = df

In [81]:
# We use dataframe when treament is aligned with new page or control
df2 = df[((df.group=='treatment') & (df.landing_page=='new_page')) | ((df.group=='control') & (df.landing_page=='old_page'))]

In [82]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [83]:
#find uniqe users
df2.user_id.nunique()

290584

b. There is one **user_id** repeated in **df2**.  What is it?

In [84]:
#find duplicate in user_id
df2.user_id[df2.user_id.duplicated()]

2893    773192
Name: user_id, dtype: int64

c. What is the row information for the repeat **user_id**? 

In [85]:
#find the row with duplicated information
df2.loc[df2.user_id.duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [86]:
#remove the duplicate
df2 = df2.drop_duplicates()

In [87]:
#check duplicates
sum(df2.duplicated())

0

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [88]:
#the probability of an individual converting regardless of the page they receive
df2['converted'].mean()

0.11959667567149027

b. Given that an individual was in the `control` group, what is the probability they converted?

In [89]:
#the probabilty if he in control group
con_gr = len(df2.query('group=="control" and converted==1'))/len(df2.query('group=="control"'))
con_gr

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [90]:
#the probabilty if he in treatment group
treatment_gr = len(df2.query('group=="treatment" and converted==1'))/len(df2.query('group=="treatment"'))
treatment_gr

0.11880724790277405

d. What is the probability that an individual received the new page?

In [91]:
# The probability of individual received a new page
len(df2.query('landing_page=="new_page"'))/len(df2.index)

0.5000636646764286

e. Consider your results from a. through d. above, and explain below whether you think there is sufficient evidence to say that the new treatment page leads to more conversions.

**Probability of individual converting given individual is in control group is 0.1203863045004612. Probability of individual converting given individual is in treatment group is 0.11880724790277405. According to the analysis this is clear that there is no more conversion between new page and old page.**

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

$$H_0: p_{new} - p_{old} <= 0$$
$$H_1: p_{new} - p_{old} > 0$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [92]:
pnew = len(df2.query( 'converted==1'))/len(df2.index)
pnew

0.11959667567149027

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [93]:
pold = len(df2.query('converted==1'))/len(df2.index)
pold

0.11959667567149027

c. What is $n_{new}$?

In [94]:
#number of queries when land page is == to new page
nnew = len(df2.query('landing_page=="new_page"'))
#print nnew
nnew

145311

d. What is $n_{old}$?

In [95]:
#number of queries when land page is == to old page
nold = len(df2.query('landing_page=="old_page"'))
#print nnew
nold

145274

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [96]:
#simulate nold trransactions with converted rate of pnew under the null
treatment_df = df2.query('group == "treatment"')
sample_new = treatment_df.sample(nnew, replace=True)
new_page_converted = sample_new['converted']
new_page_converted.mean()

0.11934402763727453

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [97]:
# simulate n_old transactions with a convert rate of p_old under the null
control_df = df2.query('group == "control"')
sample_old = control_df.sample(nold, replace=True)
old_page_converted = sample_old['converted']
old_page_converted.mean()

0.12127428170216281

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [98]:
#difference between pnew and pold
dff = new_page_converted.mean() - old_page_converted.mean()
dff

-0.0019302540648882816

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [99]:
# Create sampling distribution for difference in p_new-p_old simulated values
#bootstraping
control_conv_prob = []
treatment_conv_prob = []
p_diffs = []

# for loops are much slower than numpy functions
for _ in range(10000):
    sam_old2 = control_df.sample(nold, replace=True)
    sam_new2 = treatment_df.sample(nnew, replace=True)

    control_conversion = sam_old2['converted'].sum() / nold
    treatment_conversion = sam_new2['converted'].sum() / nnew


    
    control_conv_prob.append(control_conversion)
    treatment_conv_prob.append(treatment_conversion)
    p_diffs.append(treatment_conversion - control_conversion)
    
p_diffs = np.array(p_diffs)

KeyboardInterrupt: 

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
p_dff=np.array(p_dff)
#creating histogram
plt.hist(p_dff)
plt.title('Graph of p differents')#title of graphs
plt.xlabel('Page difference') # x-label of graphs
plt.ylabel('number of pages')

In [ ]:
p_dff=np.array(p_dff)
#creating histogram
plt.hist(p_dff)
plt.title('Graph of p differents')#title of graphs
plt.xlabel('Page difference') # x-label of graphs
plt.ylabel('number of pages')

plt.axvline(x= dff, color='r');

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
df_control = df2.query('group == "control"')
df_treatment = df2.query('group == "treatment"')

dff = df2['converted'][df2['group'] == 'treatment'].mean() - df2['converted'][df2['group'] == 'control'].mean()
dff


In [ ]:
(p_dff > dff).mean()

In [ ]:
#p value calculated here
v1 = df2[df2['landing_page'] == 'new_page']
v1=v1['converted'].mean()
v2 = df2[df2['landing_page'] == 'old_page']
v2 = v2['converted'].mean()
actual_diff = v1-v2
count = 0
for j in p_dff:
    if j> actual_diff:
        count = count+1
x = count/(len(p_dff))
print("p value is : ", x)

In [ ]:
# plot line for observed statistic
plt.hist(p_dff)
plt.axvline(x=dff, color='r', label="Obs difference")
plt.xlabel('p_diff')
plt.ylabel('Frequency')
plt.title('difference between old and new page');
plt.legend()
plt.show()

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

**The value we calculated in the previous called p-value. whem accepting null  hypothesis p-value should be greater than suggested p-value.almost 90% of the population is above the real diffrence**

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
import statsmodels.api as sm

convert_old = len(df2.query('converted==1 and landing_page=="old_page"')) 
convert_new = len(df2.query('converted==1 and landing_page=="new_page"')) 
n_old = len(df2.query('landing_page=="old_page"'))
n_new = len(df2.query('landing_page=="new_page"')) 
n_new

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
#calculating z score and p value
z_score, p_value = sm.stats.proportions_ztest([convert_old,convert_new], [n_old, n_new],alternative='smaller') 

#display z score and p value
print("The z score is:",z_score,"&&","The p value is : ", p_value)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

**The z score is less than the 95 percent confidence level's crucial threshold. As a result, we are unable to reject the null hypothesis. Therefore the conclusion is same as part j that we accept null hypothesis.**

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

**Logistic Regression.**

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
#intercept column
df2['intercept'] = 1

#Create dummy column
df2['ab_page'] = pd.get_dummies(df2['group'])['treatment']

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
import statsmodels.api as sm
model=sm.Logit(df2['converted'],df2[['intercept','ab_page']])
results=model.fit() 

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

**The related p-value for ab page is 0.19. Because 0.19 is greater than our Type I error threshold of 0.05, the null cannot be rejected.
As a result, the negative coefficient of ab page is small, and we cannot conclude that the new page has any impact on conversion rates.
Because the new page did not prove to have higher conversions, the previous page should be preserved.
This result is the same as in the previous part, however the p-value differs from the one found in Part II due to the fact that the two sections test distinct hypotheses.
Because we're doing a two-tailed test in this example, the alternative is that the new page converts at a different rate than the old page (in whichever direction). So**


f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

**Additional factor should be included in regression models because they may have an impact on conversions.
The downside is that we have no idea which way our added component will influence the outcome.
For the time being, we're simply interested in the impact of the new page on conversion rates. However, many other factors are likely to influence whether or not the user converts, such as whether they are existing customers who may have change aversion, or whether they convert due to other changes on the site or for reasons other than being presented with the new page, such as their specific customer characteristics.**



g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')

In [ ]:
## Create the necessary dummy variables
dummy_countries = pd.get_dummies(df_new['country'])
df4 = dummy_countries.join(df_new, how='inner')
df4.head()

In [ ]:
df_new['US_ab_page'] = df_new['US']*df_new['ab_page']
df_new.head()

In [101]:
log_mod1 = sm.Logit(df4['converted'], df4[['intercept', 'UK', 'CA']])
results = log_mod1.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.366638
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               302266
Model:                          Logit   Df Residuals:                   302263
Method:                           MLE   Df Model:                            2
Date:                Sat, 19 Mar 2022   Pseudo R-squ.:               1.153e-05
Time:                        21:18:44   Log-Likelihood:            -1.1082e+05
converged:                       True   LL-Null:                   -1.1082e+05
Covariance Type:            nonrobust   LLR p-value:                    0.2787
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9942      0.007   -298.076      0.000      -2.007      -1.981
UK             0.0087      0.013      0.665      0.506      -0.017       0.034
CA            -0.0354      0.026     -1.349      0.177      -0.087       0.016
==============================================================================
"""

In [102]:
log_model2 = sm.Logit(df4['converted'], df4[['intercept', 'ab_page', 'UK', 'CA']])
results = log_model2.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.366635
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               302266
Model:                          Logit   Df Residuals:                   302262
Method:                           MLE   Df Model:                            3
Date:                Sat, 19 Mar 2022   Pseudo R-squ.:               1.887e-05
Time:                        21:19:16   Log-Likelihood:            -1.1082e+05
converged:                       True   LL-Null:                   -1.1082e+05
Covariance Type:            nonrobust   LLR p-value:                    0.2424
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9871      0.009   -228.148      0.000      -2.004      -1.970
ab_page       -0.0143      0.011     -1.276      0.202      -0.036       0.008
UK             0.0086      0.013      0.662      0.508      -0.017       0.034
CA            -0.0354      0.026     -1.348      0.178      -0.087       0.016
==============================================================================
"""

**Adding ab_page to the model makes no difference, all variables are still insignificant and the null cannot be rejected.**

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
UK_newpage = df4['ab_page'] * df4['UK']
df4['UK_newpage'] = UK_newpage

In [104]:
CA_newpage = df4['ab_page'] * df4['CA']
df4['CA_newpage'] = CA_newpage
df4.head()

,CA,UK,US,country,timestamp,group,landing_page,converted,intercept,ab_page,UK_newpage,CA_newpage
user_id,,,,,,,,,,,,
630000,0,0,1,US,2017-01-19 06:26:06.548941,treatment,new_page,0,1,1,0,0
630001,0,0,1,US,2017-01-16 03:16:42.560309,treatment,new_page,1,1,1,0,0
630002,0,0,1,US,2017-01-19 19:20:56.438330,control,old_page,0,1,0,0,0
630003,0,0,1,US,2017-01-12 10:09:31.510471,treatment,new_page,0,1,1,0,0
630004,0,0,1,US,2017-01-18 20:23:58.824994,treatment,new_page,0,1,1,0,0


In [110]:
#Fit Linear Model and Obtain the result
log_model3 = sm.Logit(df4['converted'], df4[['intercept', 'UK', 'CA', 'UK_newpage', 'CA_newpage']])
results = log_model3.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.366635
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               302266
Model:                          Logit   Df Residuals:                   302261
Method:                           MLE   Df Model:                            4
Date:                Sat, 19 Mar 2022   Pseudo R-squ.:               1.936e-05
Time:                        21:26:27   Log-Likelihood:            -1.1082e+05
converged:                       True   LL-Null:                   -1.1082e+05
Covariance Type:            nonrobust   LLR p-value:                    0.3682
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9942      0.007   -298.076      0.000      -2.007      -1.981
UK             0.0080      0.017      0.469      0.639      -0.026       0.042
CA            -0.0023      0.036     -0.063      0.950      -0.073       0.069
UK_newpage     0.0012      0.022      0.055      0.956      -0.043       0.045
CA_newpage    -0.0669      0.051     -1.316      0.188      -0.167       0.033
==============================================================================
"""

In [109]:
log_model4 = sm.Logit(df4['converted'], df4[['intercept', 'ab_page', 'UK', 'CA', 'UK_newpage', 'CA_newpage']])
results = log_model4.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.366632
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               302266
Model:                          Logit   Df Residuals:                   302260
Method:                           MLE   Df Model:                            5
Date:                Sat, 19 Mar 2022   Pseudo R-squ.:               2.597e-05
Time:                        21:26:24   Log-Likelihood:            -1.1082e+05
converged:                       True   LL-Null:                   -1.1082e+05
Covariance Type:            nonrobust   LLR p-value:                    0.3307
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -1.9861      0.009   -210.438      0.000      -2.005      -1.968
ab_page       -0.0162      0.013     -1.211      0.226      -0.042       0.010
UK         -3.985e-05      0.018     -0.002      0.998      -0.036       0.036
CA            -0.0104      0.037     -0.282      0.778      -0.082       0.062
UK_newpage     0.0174      0.026      0.669      0.503      -0.034       0.069
CA_newpage    -0.0507      0.053     -0.964      0.335      -0.154       0.052
==============================================================================
"""

**The model above includes only the country of customers and no other explanatory variables. We see that these predictors are insignificant (their p-values are high), i.e. we cannot say that solely being from either UٍS or UK (as opposed to US) has a significant effect on the conversion rate.**

<a id='conclusions'></a>
## Conclusions

There are no significant p-values for any of the variables. As a result, we will fail to reject the null hypothesis and conclude that there is insufficient evidence to demonstrate that the country and page received interact to predict whether or not a user converts.
